# kernel

> IPythonKernel based on pystata

The latest documentation for implementing a wrapper kernel is [here](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html), but the current code deviates from those instructions (which call for inheriting from [kernelbase.Kernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/kernelbase.py)), instead inheriting from the IPython kernel implementation, [IPythonKernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/ipkernel.py).

In [ ]:
#| default_exp kernel
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.config import get_config, launch_stata
from nbstata.utils import print_red, ending_delimiter, is_cr_delimiter
from nbstata.stata_session import StataSession
from nbstata.magics import StataMagics
from nbstata.completions import CompletionsManager
from fastcore.basics import patch_to
from ipykernel.ipkernel import IPythonKernel
import nbstata

In [ ]:
#| export
class PyStataKernel(IPythonKernel):
    """A jupyter kernel based on pystata"""
    implementation = 'nbstata'
    implementation_version = nbstata.__version__
    language_info = {
        'name': 'stata',
        'version': '17',
        'mimetype': 'text/x-stata',
        'file_extension': '.do',
    }
    banner = "nbstata: a Jupyter kernel for Stata based on pystata"
    help_links = [
        {
            "text": "Stata Documentation",
            "url": "https://www.stata.com/features/documentation/",
        },
        {
            "text": "nbstata Help",
            "url": "https://hugetim.github.io/nbstata/",
        },
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.stata_ready = False
        self.shell.execution_count = 0
        self.starting_delimiter = None
        self.perspective_enabled = None
        try:
            self.init_stata()
        except ModuleNotFoundError as err:
            pass # wait for first do_execute so error message can be displayed under cell

In [ ]:
#| export
def _set_graph_format(graph_format):
    if graph_format != 'pystata':
        from pystata.config import set_graph_format
        set_graph_format(graph_format)

In [ ]:
#| export
@patch_to(PyStataKernel)
def init_stata(self):
    self.env = get_config()
    launch_stata(self.env['stata_dir'], self.env['edition'],
                 False if self.env['splash']=='False' else True)
    _set_graph_format(self.env['graph_format'])
    self.stata_session = StataSession()
    self.magic_handler = StataMagics()
    self.completions = CompletionsManager(self.stata_session, list(self.magic_handler.available_magics.keys()))
    self.stata_ready = True

In [ ]:
#| export
class Cell:
    """A class for managing execution of a single code cell"""
    def __init__(self, kernel, code_w_magics, silent=False):
        if kernel.env['echo'] == 'None':
            self.noecho = True
            self.echo = False
        elif kernel.env['echo'] == 'True':
            self.noecho = False
            self.echo = True
        else:
            self.noecho = False
            self.echo = False
        self.quietly = silent
        self.starting_delimiter = kernel.starting_delimiter
        self.stata_session = kernel.stata_session
        self.code = kernel.magic_handler.magic(code_w_magics, kernel, self)
    
    def run(self):
        if self.code != '':
            if self.noecho and not self.quietly:
                from nbstata.helpers import run_noecho
                run_noecho(self.code, self.starting_delimiter,
                           run_as_prog=self.stata_session.run_as_prog_with_locals)
            else:
                from pystata.stata import run
                if not is_cr_delimiter(self.starting_delimiter):
                    self.code = "#delimit;\n" + self.code
                run(self.code, quietly=self.quietly, inline=not self.quietly, echo=self.echo)

Some `Cell` functionality can be tested apart from a kernel:

In [ ]:
from fastcore.test import test_eq
from textwrap import dedent
from unittest.mock import Mock

In [ ]:
kernel1 = Mock()
kernel1.env = {'echo': 'None'}
kernel1.magic_handler = StataMagics()
kernel1.starting_delimiter = None

code_w_magics = '''disp "test output"'''
cell1 = Cell(kernel1, code_w_magics)
cell1.code

'disp "test output"'

In [ ]:
#| eval: false
launch_stata(splash=False)
kernel1.stata_session = StataSession()
cell1a = Cell(kernel1, code_w_magics)
cell1a.run()

test output


In [ ]:
code_w_magics = dedent('''\
    *%quietly
    disp "test output"
    ''')
cell2 = Cell(kernel1, code_w_magics)
test_eq(cell2.quietly, True)

In [ ]:
#| eval: false
cell2.run()

In [ ]:
kernel1.env = {'echo': 'True'}

code_w_magics = '''disp "test output"'''
cell3 = Cell(kernel1, code_w_magics)
test_eq(cell3.noecho, False)

In [ ]:
#| eval: false
cell3.run()

. disp "test output"
test output


In [ ]:
code_w_magics = dedent('''\
    *%noecho
    #delimit cr
    disp "test output"
    ''')
cell4 = Cell(kernel1, code_w_magics)
test_eq(cell4.noecho, True)

In [ ]:
#| eval: false
cell4.run()

test output


In [ ]:
#| export
_missing_stata_message = (
    "pystata path not found\n"
    "A Stata 17 installation is required to use the nbstata Stata kernel. "
    "If you already have Stata 17 installed, "
    "please specify its path in your configuration file."
)

In [ ]:
#| hide
print(_missing_stata_message)

pystata path not found
A Stata 17 installation is required to use the nbstata Stata kernel. If you already have Stata 17 installed, please specify its path in your configuration file.


In [ ]:
#| export
def _handle_stata_import_error(err, silent, execution_count):
    if not silent:
        print_red(f"ModuleNotFoundError: {_missing_stata_message}")
    return {
        "traceback": [],
        "ename": "ModuleNotFoundError",
        "evalue": _missing_stata_message,
        'status': "error",
        'execution_count': execution_count,
    }

In [ ]:
#| export
def print_stata_error(text):
    lines = text.splitlines()
    if len(lines) > 2:
        print("\n".join(lines[:-2]))
    print_red("\n".join(lines[-2:]))

In [ ]:
print_stata_error(dedent("""\
    output prior to error
    error message
    error code
    """))

output prior to error
error message
error code


In [ ]:
#| export
def _handle_stata_error(err, silent, execution_count):
    reply_content = {
        "traceback": [],
        "ename": "Stata error",
        "evalue": str(err),
    }
    if not silent:
        print_stata_error(reply_content['evalue'])
#         self.send_response(
#             self.iopub_socket,
#             "error",
#             reply_content,
#         )
    reply_content.update({
        'status': "error",
        'execution_count': execution_count,
    })
    return reply_content

In [ ]:
#| export
@patch_to(PyStataKernel)
def post_do_hook(self):
    self.stata_session.clear_suggestions()

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_execute(self, code, silent, store_history=True, user_expressions=None,
               allow_stdin=False):
    """Execute Stata code cell"""
    if not self.stata_ready:
        try:
            self.init_stata()
        except ModuleNotFoundError as err:
            return _handle_stata_import_error(err, silent, self.execution_count)
    self.shell.execution_count += 1
    _ending_delimiter = ending_delimiter(code, self.starting_delimiter)
    code_cell = Cell(self, code, silent)
    try:
        code_cell.run()
    except SystemError as err:
        return _handle_stata_error(err, silent, self.execution_count)
    post_magic_code = code_cell.code.strip()
    if _ending_delimiter == ';' and post_magic_code and post_magic_code[-1] != ';':
        print_red("Warning: Code cell (with #delimit; in effect) does not end in ';'. "
                  "Exported .do script may behave differently from notebook.")
    self.starting_delimiter = _ending_delimiter
    self.post_do_hook()
    return {
        'status': "ok",
        'execution_count': self.execution_count,
        'payload': [],
        'user_expressions': {},
    }

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_complete(self, code, cursor_pos):
    """Provide context-aware suggestions"""
    if self.stata_ready:
        cursor_start, cursor_end, matches = self.completions.do(
            code,
            cursor_pos,
            self.starting_delimiter,
        )
        return {
            'status': "ok",
            'cursor_start': cursor_start,
            'cursor_end': cursor_end,
            'metadata': {},
            'matches': matches,
        }
    else:
        return {
            'status': "ok",
            'cursor_start': cursor_pos,
            'cursor_end': cursor_pos,
            'metadata': {},
            'matches': [],
        }

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_is_complete(self, code):
    """Overrides IPythonKernel with kernelbase default"""
    return {"status": "unknown"}

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_inspect(self, code, cursor_pos, detail_level=0, omit_sections=()):
    """Overrides IPythonKernel with kernelbase default"""
    return {"status": "ok", "data": {}, "metadata": {}, "found": False}

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_history(
    self,
    hist_access_type,
    output,
    raw,
    session=None,
    start=None,
    stop=None,
    n=None,
    pattern=None,
    unique=False,
):
    """Overrides IPythonKernel with kernelbase default"""
    return {"status": "ok", "history": []}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()